In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def load_base_matrix():  
    return np.genfromtxt('matrices/riga49.csv', delimiter=',')
    
    
def generate_matrix(q):
    t = load_base_matrix()
    return np.block([[t] * q] * q)

In [5]:
q = 2**5
base_matrix = generate_matrix(q)
